# Data generation for 'The Five Monuments of Cycling'-Dashboard

This notebook creates the **data source for a Tableau dashboard** visualizing the characteristics and results of the most important 1-day races in cycling ("cycling monuments").

Running this notebook will create five files in the repository the notebook is located:
- *races.csv* (general information about the races)
- *results.csv* (rider placements of each race)
- *routes.geojson* (shape of the route for each race)
- *elevation.csv* (elevation profile for each race)
- *places.csv* (coordinates of points of interest of each race)
- *winners.csv* (riders with most monument wins in history)

In [2]:
# Import needed packages
import numpy as np
from datetime import datetime

import pandas as pd
import fiona
import geopandas as gpd

import os
import glob

In [3]:
# Generic functions for logging
def current_time():
    """
    Return current time in hh:mm:ss
    """
    current_time = datetime.now().strftime("%H:%M:%S")
    return(current_time)

def log(message):
    """
    Prints message with timestamp for logging
    """
    return(print(current_time(), '-', str(message)))

## 1. Races

Data for the eight races is entered manually. Sources: https://www.procyclingstats.com/ and https://www.wikipedia.com/.

In [4]:
# Manual entry of race data
list_races = [
    ['2022-ms-m', '2022-03-19','Milan-Sanremo', 'La Primavera', 'm', 'IT', 'Milan', 'Sanremo', 113, 1907]
    , ['2022-fl-m', '2022-04-03', 'Ronde van Vlaanderen', 'De Ronde', 'm', 'BL', 'Antwerp', 'Oudenaarde', 107, 1913]
    , ['2022-pr-m', '2022-04-17', 'Paris-Roubaix', 'Lenfer du Nord', 'm', 'FR', 'Compiègne', 'Roubaix', 119, 1896]
    , ['2022-lbl-m', '2022-04-24', 'Liège-Bastogne-Liège', 'La Doyenne', 'm', 'BL', 'Liège', 'Liège', 106, 1892]    
    , ['2022-il-m', '2022-10-08', 'Il Lombardia', 'La classica delle foglie morte', 'm', 'IT', 'Bergamo', 'Como', 116, 1905]   
    , ['2022-fl-w', '2022-04-03', 'Ronde van Vlaanderen', 'De Ronde', 'w', 'BL', 'Oudenaarde', 'Oudenaarde', 19, 2004]
    , ['2022-pr-w', '2022-04-16', 'Paris-Roubaix', "L'enfer du Nord", 'w', 'FR', 'Denain', 'Roubaix', 2, 2021]
    , ['2022-lbl-w', '2022-04-24', 'Liège-Bastogne-Liège', 'La Doyenne', 'w', 'BL', 'Bastogne', 'Liège', 6, 2017]    
]
  
# Create the DataFrame df_races
df_races = pd.DataFrame(list_races
                        , columns=['id', 'date', 'race', 'alias', 'gender', 'country', 'start', 'finish', 'number', 'established']
                       )

# Inspect result
print(df_races.head())

           id        date                  race  \
0   2022-ms-m  2022-03-19         Milan-Sanremo   
1   2022-fl-m  2022-04-03  Ronde van Vlaanderen   
2   2022-pr-m  2022-04-17         Paris-Roubaix   
3  2022-lbl-m  2022-04-24  Liège-Bastogne-Liège   
4   2022-il-m  2022-10-08          Il Lombardia   

                            alias gender country      start      finish  \
0                    La Primavera      m      IT      Milan     Sanremo   
1                        De Ronde      m      BL    Antwerp  Oudenaarde   
2                  Lenfer du Nord      m      FR  Compiègne     Roubaix   
3                      La Doyenne      m      BL      Liège       Liège   
4  La classica delle foglie morte      m      IT    Bergamo        Como   

   number  established  
0     113         1907  
1     107         1913  
2     119         1896  
3     106         1892  
4     116         1905  


In [23]:
# Save to .csv
df_races.to_csv('races.csv', index=False)

## 1. Races

Data for riders placed in the top 10 are entered manually. Source: https://www.procyclingstats.com/.

In [5]:
# Manual entry of top 10 results for all monuments retrieved from procyclingstats

list_results = [
    ['2022-ms-m', 1, 'Matej Mohorič', 'SLO', '6:27:49']
    , ['2022-ms-m', 2, 'Anthony Turgis', 'FR', '6:27:51']
    , ['2022-ms-m', 3, 'Mathieu van der Poel', 'NL', '6:27:51']
    , ['2022-ms-m', 4, 'Michael Matthews', 'AUS', '6:27:51']
    , ['2022-ms-m', 5, 'Tadej Pogačar', 'SLO', '6:27:51']
    , ['2022-ms-m', 6, 'Mads Pedersen', 'DK', '6:27:51']
    , ['2022-ms-m', 7, 'Søren Kragh Andersen', 'DK', '6:27:51']
    , ['2022-ms-m', 8, 'Wout van Aart', 'BL', '6:27:51']
    , ['2022-ms-m', 9, 'Jan Tratnik', 'SLO', '6:27:53']
    , ['2022-ms-m', 10, 'Arnaud Démare', 'FR', '6:27:59']
    
    , ['2022-fl-m', 1, 'Mathieu van der Poel', 'NL', '6:18:30']
    , ['2022-fl-m', 2, 'Dylan van Baarle', 'NL', '6:18:30']
    , ['2022-fl-m', 3, 'Valentin Madouas', 'FR', '6:18:30']
    , ['2022-fl-m', 4, 'Tadej Pogačar', 'SLO', '6:18:30']
    , ['2022-fl-m', 5, 'Stefan Küng', 'SUI', '6:18:32']
    , ['2022-fl-m', 6, 'Dylan Teuns', 'BL', '6:18:32']
    , ['2022-fl-m', 7, 'Fred Wright', 'GB', '6:18:41']
    , ['2022-fl-m', 8, 'Mads Pedersen', 'DK', '6:19:18']
    , ['2022-fl-m', 9, 'Christophe Laporte', 'FR', '6:19:18']
    , ['2022-fl-m', 10, 'Alexander Kristoff', 'NOR', '6:19:18']     
    
    , ['2022-fl-w', 1, 'Lotte Kopecky', 'BL', '4:11:21']
    , ['2022-fl-w', 2, 'Annemiek van Vleuten', 'NL', '4:11:21']
    , ['2022-fl-w', 3, 'C. van den Broek-Blaak', 'NL', '4:11:23']
    , ['2022-fl-w', 4, 'Arlenis Sierra', 'CU', '4:12:01']
    , ['2022-fl-w', 5, 'Marlen Reusser', 'SUI', '4:12:01']
    , ['2022-fl-w', 6, 'Cecilie U. Ludwig', 'DK', '4:12:01']
    , ['2022-fl-w', 7, 'Grace Brown', 'AUS', '4:12:01']
    , ['2022-fl-w', 8, 'Katarzyna Niewiadoma', 'PL', '4:12:01']
    , ['2022-fl-w', 9, 'Brodie Chapman', 'AUS', '4:12:03']
    , ['2022-fl-w', 10, 'Marta Bastianelli', 'IT', '4:12:31']   
    
    , ['2022-pr-m', 1, 'Dylan van Baarle', 'NL', '5:37:00']
    , ['2022-pr-m', 2, 'Wout van Aert', 'BL', '5:38:47']
    , ['2022-pr-m', 3, 'Stefan Küng', 'SUI', '5:38:47']
    , ['2022-pr-m', 4, 'Tom Devriendt', 'BL', '5:38:47']
    , ['2022-pr-m', 5, 'Matej Mohorič', 'SLO', '5:38:47']
    , ['2022-pr-m', 6, 'Adrien Petit', 'FR', '5:39:27']
    , ['2022-pr-m', 7, 'Jasper Stuyven', 'BL', '5:39:27']
    , ['2022-pr-m', 8, 'Laurent Pichon', 'FR', '5:39:27']
    , ['2022-pr-m', 9, 'Mathieu van der Poel', 'NL', '5:39:34']
    , ['2022-pr-m', 10, 'Yves Lampaert', 'BL', '5:39:59']     
    
    , ['2022-pr-w', 1, 'Elisa Longo Borghini', 'IT', '3:10:54']
    , ['2022-pr-w', 2, 'Lotte Kopecky', 'BL', '3:11:17']
    , ['2022-pr-w', 3, 'Lucinda Brand', 'NL', '3:11:17']
    , ['2022-pr-w', 4, 'Elise Chabbey', 'SUI', '3:11:17']
    , ['2022-pr-w', 5, 'Marta Cavalli', 'IT', '3:11:17']
    , ['2022-pr-w', 6, 'Floortje Mackaij', 'NL', '3:11:17']
    , ['2022-pr-w', 7, 'Ellen van Dijk', 'NL', '3:11:17']
    , ['2022-pr-w', 8, 'C. van den Broek-Blaak', 'NL', '3:11:26']
    , ['2022-pr-w', 9, 'Pfeiffer Georgi', 'GB', '3:13:16']
    , ['2022-pr-w', 10, 'Sandra Alonso', 'ESP', '3:13:16']  
    
    , ['2022-lbl-m', 1, 'Remco Evenepoel', 'BL', '6:12:38']
    , ['2022-lbl-m', 2, 'Quinten Hermans', 'BL', '6:13:26']
    , ['2022-lbl-m', 3, 'Wout van Aert', 'BL', '6:13:26']
    , ['2022-lbl-m', 4, 'Daniel F. Martínez', 'COL', '6:13:26']
    , ['2022-lbl-m', 5, 'Sergio Higuita', 'COL', '6:13:26']
    , ['2022-lbl-m', 6, 'Dylan Teuns', 'BL', '6:13:26']
    , ['2022-lbl-m', 7, 'Alejandro Valverde', 'ESP', '6:13:26']
    , ['2022-lbl-m', 8, 'Neilson Powless', 'US', '6:13:26']
    , ['2022-lbl-m', 9, 'Marc Hirschi', 'SUI', '6:13:26']
    , ['2022-lbl-m', 10, 'Michael Woods', 'CAN', '6:13:26']     
    
    , ['2022-lbl-w', 1, 'Annemiek van Vleuten', 'NL', '3:52:32']
    , ['2022-lbl-w', 2, 'Grace Brown', 'AUS', '3:53:15']
    , ['2022-lbl-w', 3, 'Demi Vollering', 'NL', '3:53:15']
    , ['2022-lbl-w', 4, 'Ashleigh Moolman', 'SA', '3:53:15']
    , ['2022-lbl-w', 5, 'Elisa Longo Borghini', 'IT', '3:53:15']
    , ['2022-lbl-w', 6, 'Marta Cavalli', 'IT', '3:53:19']
    , ['2022-lbl-w', 7, 'Arlenis Sierra', 'CU', '3:54:30']
    , ['2022-lbl-w', 8, 'Liane Lippert', 'DE', '3:54:30']
    , ['2022-lbl-w', 9, 'Katarzyna Niewiadoma', 'PL', '3:54:30']
    , ['2022-lbl-w', 10, 'Amanda Spratt', 'AUS', '3:54:30']   
    
    , ['2022-il-m', 1, 'Tadej Pogačar', 'SLO', '6:21:22']
    , ['2022-il-m', 2, 'Enric Mas', 'ESP', '6:21:22']
    , ['2022-il-m', 3, 'Mikel Landa', 'ESP', '6:21:32']
    , ['2022-il-m', 4, 'Sergio Higuita', 'COL', '6:22:14']
    , ['2022-il-m', 5, 'Carlos Rodríguez', 'ESP', '6:22:14']
    , ['2022-il-m', 6, 'Alejandro Valverde', 'ESP', '6:22:46']
    , ['2022-il-m', 7, 'Bauke Mollema', 'NL', '6:22:46']
    , ['2022-il-m', 8, 'Rudy Molard', 'FR', '6:22:46']    
    , ['2022-il-m', 9, 'Romain Bardet', 'FR', '6:22:46']
    , ['2022-il-m', 10, 'Adam Yates', 'GB', '6:22:46']
]

  
# Create the DataFrame df_results
df_results = pd.DataFrame(list_results
                        , columns=['id', 'placement', 'rider', 'nationality', 'time']
                       )

# Inspect data
print(df_results)

           id  placement                 rider nationality     time
0   2022-ms-m          1         Matej Mohorič         SLO  6:27:49
1   2022-ms-m          2        Anthony Turgis          FR  6:27:51
2   2022-ms-m          3  Mathieu van der Poel          NL  6:27:51
3   2022-ms-m          4      Michael Matthews         AUS  6:27:51
4   2022-ms-m          5         Tadej Pogačar         SLO  6:27:51
..        ...        ...                   ...         ...      ...
75  2022-il-m          6    Alejandro Valverde         ESP  6:22:46
76  2022-il-m          7         Bauke Mollema          NL  6:22:46
77  2022-il-m          8           Rudy Molard          FR  6:22:46
78  2022-il-m          9         Romain Bardet          FR  6:22:46
79  2022-il-m         10            Adam Yates          GB  6:22:46

[80 rows x 5 columns]


In [6]:
# Save as .csv
df_results.to_csv('results.csv', index=False)

## 3. Elevation profiles

In the dashboard the elevation profiles of each race should be visualized in graph displaying kilometers on the x-axis and elevation in meters on the y-axis. The needed data is calculated from the .gpx-tracks of each race. I downloaded the .gpx-files from https://www.cyclingstages.com/ manually. They need to be placed into the folder *gpx_files*. The caluclation used in this notebook is simplified and my not represent the exact distance and elevation of each race. 

In [8]:
# Create folder for .gpx-files

folder_name = 'gpx_files'

if os.path.exists(folder_name):
    log('Folder ' + folder_name + ' allready exists!')
else:
    os.makedirs(folder_name)
    log('Folder ' + folder_name + ' created.')

13:19:07 - Folder gpx_files allready exists!


In [9]:
# Define path to .gpx-files

gpx_files = glob.glob(folder_name+"/*.gpx")
print(gpx_files)

['gpx_files/2022_w_flanders_route.gpx', 'gpx_files/2022_m_lombardia_route.gpx', 'gpx_files/2022_w_lbl_route.gpx', 'gpx_files/2022_m_flanders_route.gpx', 'gpx_files/2022_m_lbl_route.gpx', 'gpx_files/2022_m_roubaix_route.gpx', 'gpx_files/2022_m_ms_route.gpx', 'gpx_files/2022_w_roubaix_route.gpx']


Each track consists of hundreds of coordinates with corrosponding elevation. We need to loop over them, extract them and store them in a DataFrame. In a second step distance between coordinates and elevation change can be calculated through window function. Finally the data is aggregated to only full kilometers (e.g. a race with 100km distance has 100 rows in the final .csv-file.

In [11]:
gdf_elevation = gpd.GeoDataFrame()

i=1
for file in gpx_files:
    log(('Parsing .gpx file '+ str(i) + '/' + str(len(gpx_files)) + ': ' + file))
    # Extract track point layer
    d_track_points = fiona.open(file, layer='track_points')
    
    # Loop over track points within the layer
    df_track_points = pd.DataFrame()
    for pt in d_track_points:
        df = pd.DataFrame({'file_name': file
                           , 'ele': [pt['properties']['ele']]
                           , 'lat': [pt['geometry']['coordinates'][1]]                       
                           , 'lon': [pt['geometry']['coordinates'][0]]
                          })
        df_track_points = pd.concat([df_track_points,df], axis=0, join='outer')
    
    # Convert extracted data into GeoDataFrame and only keep data for first full km
    gdf = gpd.GeoDataFrame(df_track_points
                           , geometry=gpd.points_from_xy(df_track_points.lon, df_track_points.lat)
                           , crs="EPSG:4326"
                          )
    
    # Calculate distance and km
    gdf = gdf.to_crs('EPSG:3310',inplace=False)
    gdf['distance'] = gdf.distance(gdf.shift())
    gdf['km'] = gdf['distance'].cumsum()/1000
    
    # Aggegate to full km and keep coordinate with max elevation
    gdf['km'] = gdf['km'].apply(np.floor)
    gdf['km'] = gdf['km'].fillna(0)
    gdf = gdf.sort_values(by='ele', ascending=False).groupby('km', as_index=False).first()
    
    # Calculate elevation change
    gdf['ele_change'] = gdf['ele'].diff()
    gdf['ele_change'] = gdf['ele_change'].fillna(0)
    
    # Append to main GeoDataFrame
    gdf_elevation = pd.concat([gdf_elevation, gdf], ignore_index=True, axis=0, join='outer')  
    
    i = i+1

# Generate ID from file_name
gdf_elevation.loc[(gdf_elevation['file_name'].str.contains("m_flanders")),'id'] = '2022-fl-m'
gdf_elevation.loc[(gdf_elevation['file_name'].str.contains("w_flanders")),'id'] = '2022-fl-w'
gdf_elevation.loc[(gdf_elevation['file_name'].str.contains("m_lombardia")),'id'] = '2022-il-m'
gdf_elevation.loc[(gdf_elevation['file_name'].str.contains("m_lbl")),'id'] = '2022-lbl-m'
gdf_elevation.loc[(gdf_elevation['file_name'].str.contains("w_lbl")),'id'] = '2022-lbl-w'
gdf_elevation.loc[(gdf_elevation['file_name'].str.contains("m_roubaix")),'id'] = '2022-pr-m'
gdf_elevation.loc[(gdf_elevation['file_name'].str.contains("w_roubaix")),'id'] = '2022-pr-w'
gdf_elevation.loc[(gdf_elevation['file_name'].str.contains("m_ms")),'id'] = '2022-ms-m'

13:25:17 - Parsing .gpx file 1/8: gpx_files/2022_w_flanders_route.gpx


/home/lennart/anaconda3/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as NAD83 / California Albers (the single non-null crs provided).
  warnings.warn(


13:25:23 - Parsing .gpx file 2/8: gpx_files/2022_m_lombardia_route.gpx


/home/lennart/anaconda3/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as NAD83 / California Albers (the single non-null crs provided).
  warnings.warn(


13:25:43 - Parsing .gpx file 3/8: gpx_files/2022_w_lbl_route.gpx


...

/home/lennart/anaconda3/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as NAD83 / California Albers (the single non-null crs provided).
  warnings.warn(


13:26:21 - Parsing .gpx file 7/8: gpx_files/2022_m_ms_route.gpx


/home/lennart/anaconda3/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as NAD83 / California Albers (the single non-null crs provided).
  warnings.warn(


13:26:36 - Parsing .gpx file 8/8: gpx_files/2022_w_roubaix_route.gpx


/home/lennart/anaconda3/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as NAD83 / California Albers (the single non-null crs provided).
  warnings.warn(


In [12]:
# Filter columns
df_elevation = gdf_elevation[['id','km','ele','ele_change','lat','lon']]

# Inspect data
print(df_elevation)
print(df_elevation.groupby('id').last())

             id     km   ele  ele_change       lat      lon
0     2022-fl-w    0.0  97.0         0.0  50.83005  3.67937
1     2022-fl-w    1.0  96.0        -1.0  50.83118  3.68089
2     2022-fl-w    2.0  86.0       -10.0  50.83794  3.68829
3     2022-fl-w    3.0  62.0       -24.0  50.84469  3.69739
4     2022-fl-w    4.0  56.0        -6.0  50.85203  3.72007
...         ...    ...   ...         ...       ...      ...
1748  2022-pr-w  121.0  50.0        14.0  50.66141  3.18370
1749  2022-pr-w  122.0  50.0         0.0  50.66254  3.18293
1750  2022-pr-w  123.0  37.0       -13.0  50.67217  3.19075
1751  2022-pr-w  124.0  35.0        -2.0  50.67755  3.19917
1752  2022-pr-w  125.0  29.0        -6.0  50.67741  3.20533

[1753 rows x 6 columns]
               km    ele  ele_change       lat      lon
id                                                     
2022-fl-m   273.0   11.0        -3.0  50.84013  3.58989
2022-fl-w   157.0   12.0        -1.0  50.84054  3.59262
2022-il-m   252.0  206.0       

In [13]:
# Save data to .csv
df_elevation.to_csv('elevation.csv', index=False)

## 4. Route shapefiles

Contrary to the elevation the route shapes can be extracted quite easily from the .gpx-files.

In [14]:
folder = "gpx_files/"
gdf_routes = gpd.GeoDataFrame()

for file in os.listdir(folder):
    if file.endswith(('.gpx')):
        try:
            log('Extracting ' + folder + file)
            gdf = gpd.read_file(folder + file, layer='tracks')
            gdf = gdf[['geometry']]
            gdf['file_name'] = file
            gdf_routes = gpd.GeoDataFrame( pd.concat( [gdf_routes, gdf], ignore_index=True )) 
            
        except:
            log("Error extracting" + folder + file)
            
# Define ID from file_name
gdf_routes.loc[(gdf_routes['file_name'].str.contains("m_flanders")),'id'] = '2022-fl-m'
gdf_routes.loc[(gdf_routes['file_name'].str.contains("w_flanders")),'id'] = '2022-fl-w'
gdf_routes.loc[(gdf_routes['file_name'].str.contains("m_lombardia")),'id'] = '2022-il-m'
gdf_routes.loc[(gdf_routes['file_name'].str.contains("m_lbl")),'id'] = '2022-lbl-m'
gdf_routes.loc[(gdf_routes['file_name'].str.contains("w_lbl")),'id'] = '2022-lbl-w'
gdf_routes.loc[(gdf_routes['file_name'].str.contains("m_roubaix")),'id'] = '2022-pr-m'
gdf_routes.loc[(gdf_routes['file_name'].str.contains("w_roubaix")),'id'] = '2022-pr-w'
gdf_routes.loc[(gdf_routes['file_name'].str.contains("m_ms")),'id'] = '2022-ms-m'

# Filter Columns
gdf_routes = gdf_routes[['id','geometry']]

13:47:07 - Extracting gpx_files/2022_w_flanders_route.gpx
13:47:08 - Extracting gpx_files/2022_m_lombardia_route.gpx
13:47:08 - Extracting gpx_files/2022_w_lbl_route.gpx
13:47:08 - Extracting gpx_files/2022_m_flanders_route.gpx
13:47:08 - Extracting gpx_files/2022_m_lbl_route.gpx
13:47:08 - Extracting gpx_files/2022_m_roubaix_route.gpx
13:47:08 - Extracting gpx_files/2022_m_ms_route.gpx
13:47:08 - Extracting gpx_files/2022_w_roubaix_route.gpx


In [15]:
# Inspect result
print(gdf_routes)

           id                                           geometry
0   2022-fl-w  MULTILINESTRING ((3.66731 50.83107, 3.66737 50...
1   2022-il-m  MULTILINESTRING ((9.71168 45.68780, 9.71184 45...
2  2022-lbl-w  MULTILINESTRING ((5.72941 50.02061, 5.72989 50...
3   2022-fl-m  MULTILINESTRING ((4.34544 51.20171, 4.34521 51...
4  2022-lbl-m  MULTILINESTRING ((5.61442 50.60436, 5.61441 50...
5   2022-pr-m  MULTILINESTRING ((2.89534 49.44582, 2.89534 49...
6   2022-ms-m  MULTILINESTRING ((9.16173 45.41251, 9.16171 45...
7   2022-pr-w  MULTILINESTRING ((3.38919 50.34771, 3.38753 50...


In [16]:
# Save line-shapes as .geojson
gdf_routes.to_file('routes.geojson')

## 5. Places

In [18]:
list_places = [
    ['2022-ms-m', 'Velodromo Vigorelli, Milan', 9.16173, 45.41251, 'Via Roma, Sanremo', 7.77624, 43.81599, 'Poggio',10.18359612565498,42.78671980001402]
    , ['2022-fl-m', 'Antwerp', 4.34544, 51.20171, 'Oudenaarde', 3.59005, 50.84015, 'Paterberg', 3.548710, 50.782045]
    , ['2022-pr-m', 'Compiègne', 2.89534, 49.44582, 'Roubaix Velodrome', 3.20601, 50.67783, "Trouée d’Arenberg", 3.410819, 50.401088]
    , ['2022-lbl-m', 'Liège', 5.61442, 50.60436, 'Liège', 5.59222, 50.61842, 'Côte de la Redoute', 5.707572539502681, 50.4889644028033]    
    , ['2022-il-m', 'Bergamo', 9.71168, 45.68780, 'Como', 9.08068, 45.81331, 'Madonna del Ghisallo', 9.24424012485198, 45.928812232147465]   
    , ['2022-fl-w', 'Horebeeke', 3.66731, 50.83107, 'Oudenaarde', 3.59277, 50.84057, 'Paterberg', 3.548710, 50.782045]
    , ['2022-pr-w', 'Denain', 3.66731, 50.83107, 'Roubaix Velodrome', 3.20628, 50.67799, "Carrefour de l’Arbre", 3.2297801106725164, 50.589717314595184, ]
    , ['2022-lbl-w', 'Bastogne', 3.38919, 50.34771, 'Liège', 5.59059, 50.61922, 'Côte de la Redoute', 5.707572539502681, 50.4889644028033] 
]
  
# Create the DataFrame df_races
df_places = pd.DataFrame(list_places
                        , columns=['id', 'start_name', 'start_lat', 'start_lon', 'finish_name', 'finish_lat', 'finish_lon', 'highlight_name', 'highlight_lat', 'highlight_lon']
                       )
  
print(df_places)

           id                  start_name  start_lat  start_lon  \
0   2022-ms-m  Velodromo Vigorelli, Milan    9.16173   45.41251   
1   2022-fl-m                     Antwerp    4.34544   51.20171   
2   2022-pr-m                   Compiègne    2.89534   49.44582   
3  2022-lbl-m                       Liège    5.61442   50.60436   
4   2022-il-m                     Bergamo    9.71168   45.68780   
5   2022-fl-w                   Horebeeke    3.66731   50.83107   
6   2022-pr-w                      Denain    3.66731   50.83107   
7  2022-lbl-w                    Bastogne    3.38919   50.34771   

         finish_name  finish_lat  finish_lon        highlight_name  \
0  Via Roma, Sanremo     7.77624    43.81599                Poggio   
1         Oudenaarde     3.59005    50.84015             Paterberg   
2  Roubaix Velodrome     3.20601    50.67783     Trouée d’Arenberg   
3              Liège     5.59222    50.61842    Côte de la Redoute   
4               Como     9.08068    45.81331  

In [19]:
# Save to .csv
df_places.to_csv('places.csv', index=False)

## 6. Most wins

In [20]:
list_winners = [
    ['m','Eddy Merckx','BL','N',7,2,3,5,2,1966,1976]
    , ['m','Roger De Vlaeminck','BL','N',8,1,4,1,2,1970,1979]
    , ['m','Costante Girardengo','IT','N',6,0,0,0,3,1918,1928]
    , ['m','Fausto Coppi','IT','N',3,0,1,0,5,1946,1954]
    , ['m','Sean Kelly','IR','N',2,0,2,2,3,1983,1992]
    , ['m','Rik van Loy','BL','N',1,2,3,1,1,1958,1965]
    , ['m','Gino Bartali','IT','N',4,0,0,0,3,1939,1950]
    , ['m','Tom Boonen','BL','N',0,3,4,0,0,2005,2012]
    , ['m','Fabian Cancellara','SUI','N',1,3,3,0,0,2006,2014]
    , ['m','Henri Pélissier','FR','N',1,0,2,0,3,1911,1921]

    , ['w','Annemiek van Vleuten','NL','Y',0,2,0,2,0,2011,2022]
    , ['w','Zoulfia Zabirova','RU','N',2,1,0,0,0,2002,2004]
    , ['w','Mirjam Melchers-van Poppel','NL','N',1,2,0,0,0,2002,2006]
    , ['w','Anna van der Breggen','NL','N',0,1,0,2,0,2017,2018]
    , ['w','Lizzie Deignan','GB','Y',0,1,1,1,0,2016,2021]
    , ['w','Judith Arndt','DE','N',0,2,0,0,0,2008,2012]
    , ['w','Elisa Longo Borghini','IT','Y',0,1,1,0,0,2015,2022]

]
  
# Create the DataFrame df_races
df_winners = pd.DataFrame(list_winners
                        , columns=['gender', 'rider', 'nationality','active_flg','wins_ms','wins_fl','wins_pr','wins_lbl','wins_il','win_first','win_last']
                       )
  
print(df_winners)

   gender                       rider nationality active_flg  wins_ms  \
0       m                 Eddy Merckx          BL          N        7   
1       m          Roger De Vlaeminck          BL          N        8   
2       m         Costante Girardengo          IT          N        6   
3       m                Fausto Coppi          IT          N        3   
4       m                  Sean Kelly          IR          N        2   
5       m                 Rik van Loy          BL          N        1   
6       m                Gino Bartali          IT          N        4   
7       m                  Tom Boonen          BL          N        0   
8       m           Fabian Cancellara         SUI          N        1   
9       m             Henri Pélissier          FR          N        1   
10      w        Annemiek van Vleuten          NL          Y        0   
11      w            Zoulfia Zabirova          RU          N        2   
12      w  Mirjam Melchers-van Poppel          NL  

In [21]:
# Save to .csv
df_winners.to_csv('winners.csv', index=False)

### You should now find 5 .csv-files and 1 .geojson in the same repository this notebook is stored.

